In [38]:
STEERING_VECTOR_FILE_PATH = "steering_vector_unnormalized_meandiff_normalized_nuisance_normalized_steering.pkl"
PROMPTS_FILE_PATH_FULL = "vector_steering_samples_full_balanced.json"

In [39]:
import torch
import pprint
import pickle
import os
import dotenv
dotenv.load_dotenv()
from transformers import AutoModelForCausalLM, AutoTokenizer
from enhanced_hooking import (
    get_activations,
    add_activations_and_generate,
    zeroout_projections_and_generate,
    clear_hooks,
)
from transformers import logging

logging.set_verbosity_warning()


In [40]:
import json
data = json.load(open("vector_steering_samples_full_balanced.json","r"))

samples = [data['neg'][i]['forward_prompt'] for i in range(len(data['neg']))]

In [4]:
device = torch.device("cuda:0")
print(device)
model_name = "meta-llama/Llama-3.1-8B-Instruct"
# model_name = "mistralai/Mistral-7B-Instruct-v0.3"
model      = AutoModelForCausalLM.from_pretrained(
                 model_name,
                 torch_dtype=torch.float16,
                 token=os.getenv("HF_TOKEN"))
tok        = AutoTokenizer.from_pretrained(model_name)
tok.pad_token = tok.eos_token
model.to(0)


cuda:0


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  3.07it/s]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

In [41]:

with open(STEERING_VECTOR_FILE_PATH, "rb") as f:
    steering_vectors = pickle.load(f)

print(type(steering_vectors))
pprint.pprint(steering_vectors)

specificpos_layer_activations = {}

for layer_idx, vec_list in steering_vectors.items():
    
    vec = vec_list[0]
    vec *= 2 #downweight
    # Make sure dtype/device matches the model you’ll run on
    vec = vec.to(dtype=model.dtype, device=next(model.parameters()).device)

    # Tell the hook: “for this layer, add vec to position -1 (the last token)”
    specificpos_layer_activations[layer_idx] = { -1: vec }


<class 'collections.defaultdict'>
defaultdict(<class 'list'>,
            {0: [tensor([ 0.0078, -0.0037,  0.0276,  ...,  0.0225,  0.0041,  0.0072]),
                 tensor([ 0.0078,  0.0188,  0.0065,  ..., -0.0050,  0.0173,  0.0321]),
                 tensor([-0.0138,  0.0052,  0.0064,  ..., -0.0051,  0.0042, -0.0293]),
                 tensor([ 0.0140,  0.0141,  0.0118,  ..., -0.0170, -0.0032,  0.0057]),
                 tensor([ 0.0078,  0.0219, -0.0058,  ...,  0.0005, -0.0127,  0.0266]),
                 tensor([ 0.0073,  0.0089, -0.0011,  ...,  0.0390, -0.0225, -0.0121]),
                 tensor([ 0.0187,  0.0192, -0.0001,  ...,  0.0282,  0.0071,  0.0119]),
                 tensor([-0.0031,  0.0173, -0.0068,  ...,  0.0531, -0.0190, -0.0152]),
                 tensor([ 0.0051,  0.0138, -0.0091,  ...,  0.0134, -0.0205,  0.0088]),
                 tensor([ 0.0225,  0.0160, -0.0072,  ...,  0.0178, -0.0228, -0.0075])],
             1: [tensor([ 0.0114, -0.0037,  0.0186,  ...,  0.0019, 

In [42]:
import json
prompts = []
with open(PROMPTS_FILE_PATH_FULL, 'r') as f:
    prompts_raw = json.load(f)
    # pprint.pprint(prompts_raw)
    # for _ in prompts_raw['pos']:
    #     prompts.append(_['forward_prompt'])
    #     prompts.append(_['backward_prompt'])
    # for _ in prompts_raw['neg']:
    #     prompts.append(_['forward_prompt'])
    #     prompts.append(_['backward_prompt'])
    # json.dump({"prompts": prompts}, open("prompts_full.json", "w"))


In [43]:
sampling_kwargs={"use_cache": True, "pad_token_id": tok.eos_token_id, "max_new_tokens": 10, "do_sample": False}
tok.padding_side = "left"

{'pos': {}}

In [ ]:
import random
from tqdm import tqdm
thinker = []
layers = range(14, 17)
mults = [3, 6, 10, 18]
results = {}
for key in ["pos"]:
    assert key not in results
    results[key] = {}
    for record in tqdm(prompts_raw[key]):
        assert record['key'] not in results[key]
        results[key][record['key']] = {}
        if not isinstance(record, dict):
            continue

        # ← CHANGED: randomly pick forward vs. backward 50/50
        side = 'forward_prompt'

        # ← NEW: print which prompt side we’re using, plus the exact text
        print(f"\n=== Using prompt side: {side} ===")
        #print(f"Prompt text:\n{prompt}\n")
        prompt = record[side]
        # Tokenize only once for this (side, record) combination
        prompt = tok.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False)
        tokens = tok(prompt, return_tensors="pt").to(model.device)

        for layer in layers:
            assert layer not in results[key]
            results[key][record['key']][layer] = {}
            base_vec = steering_vectors[layer][0].to(
                dtype=model.dtype, device=model.device
            )

            for mult in mults:
                assert mult not in results[key][record['key']][layer]
                results[key][record['key']][layer][mult] = {}
                payload = {}
                #print(f"Mult {mult}")
                scaled_vec = base_vec * mult
                pos_vecs = {layer: {-1: scaled_vec}}
                neg_vecs = {layer: {-1: -scaled_vec}}
                # ─── POS ───
                clear_hooks(model)
                ids_pos = add_activations_and_generate(
                    model,
                    tokens,
                    specificpos_layer_activations=pos_vecs,
                    continuouspos_layer_activations={},
                    sampling_kwargs=sampling_kwargs,
                    add_at="end"
                )
                txt_pos = tok.decode(
                    ids_pos[0, tokens["input_ids"].shape[1]:],
                    skip_special_tokens=True
                ).strip()
                payload['pos'] = txt_pos.replace("assistant\n\n", "")
                # print("Pos")
                # print(txt_pos.replace("assistant\n\n", ""))

                # ─── NEG ───
                clear_hooks(model)
                ids_neg = add_activations_and_generate(
                    model,
                    tokens,
                    specificpos_layer_activations=neg_vecs,
                    continuouspos_layer_activations={},
                    sampling_kwargs=sampling_kwargs,
                    add_at="end"
                )
                txt_neg = tok.decode(
                    ids_neg[0, tokens["input_ids"].shape[1]:],
                    skip_special_tokens=True
                ).strip()
                payload['neg'] = txt_neg.replace("assistant\n\n", "")
                # print("Neg")
                # print(txt_neg.replace("assistant\n\n", ""))


                # ─── NONE / Plain Model ───
                clear_hooks(model)
                ids_none = model.generate(**tokens, **sampling_kwargs)
                txt_none = tok.decode(
                    ids_none[0, tokens["input_ids"].shape[1]:],
                    skip_special_tokens=True
                ).strip()
                payload['none'] = txt_none.replace("assistant\n\n", "")

                # print("Plain")
                # print(txt_none.replace("assistant\n\n", ""))
                assistant = "assistant\n\n"
                results[key][record['key']][layer][mult] = payload
        json.dump(results, open("steering_graph_results.json","w"))



  0%|          | 0/155 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



=== Using prompt side: forward_prompt ===


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


=== Using prompt side: forward_prompt ===


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.

...

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


=== Using prompt side: forward_prompt ===


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


=== Using prompt side: forward_prompt ===


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.

In [80]:
json.dump(results, open("steering_graph_results_shit.json","w"))


In [74]:
thinker

['Mult 0 Layer 10 + 1 - 1 none 1',
 'Mult 3 Layer 10 + 1 - 1 none 1',
 'Mult 4 Layer 10 + 1 - 1 none 1',
 'Mult 5 Layer 10 + 1 - 1 none 1',
 'Mult 6 Layer 10 + 1 - 1 none 1',
 'Mult 10 Layer 10 + 1 - is the better summary. none 1',
 'Mult 14 Layer 10 + 1 - , none 1',
 'Mult 0 Layer 11 + 1 - 1 none 1',
 'Mult 3 Layer 11 + 1 - 1 none 1',
 'Mult 4 Layer 11 + 1 - 1 none 1',
 'Mult 5 Layer 11 + 1 - 1 none 1',
 'Mult 6 Layer 11 + 1 - 1 none 1',
 'Mult 10 Layer 11 + 1 - 1 none 1',
 'Mult 14 Layer 11 + 1 - , none 1',
 'Mult 0 Layer 12 + 1 - 1 none 1',
 'Mult 3 Layer 12 + 1 - 1 none 1',
 'Mult 4 Layer 12 + 1 - 1 none 1',
 'Mult 5 Layer 12 + 1 - 1 none 1',
 'Mult 6 Layer 12 + 1 - 1 none 1',
 'Mult 10 Layer 12 + 1 - and none 1',
 'Mult 14 Layer 12 + . - , none 1',
 'Mult 0 Layer 13 + 1 - 1 none 1',
 'Mult 3 Layer 13 + 1 - 1 none 1',
 'Mult 4 Layer 13 + 1 - 1 none 1',
 'Mult 5 Layer 13 + 1 - 1 none 1',
 'Mult 6 Layer 13 + 1 - 1 none 1',
 'Mult 10 Layer 13 + 1 - 1 none 1',
 'Mult 14 Layer 13 + 1 - 

In [181]:
# For a single prompt example. Can scale to multiple prompts
results = {'results': []}
for key in ['neg']:
    records = prompts_raw[key]
    for record in records:
        for pas in ['backward_prompt']:
            prompt = record[pas]
            #print(prompt)
            prompt = tok.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False)
            #print(prompt)
            tokens = tok(prompt, return_tensors="pt").to(model.device)   
            # Generate pos steering outputs
            clear_hooks(model)
            generated_ids = add_activations_and_generate(
                model,
                tokens,
                specificpos_layer_activations = specificpos_layer_activations,
                continuouspos_layer_activations = {},      # or the dict above
                sampling_kwargs = sampling_kwargs,
                add_at = "end"        # "end" means after each transformer block forward pass
            )

            # decode to text
            generated_text_pos = tok.batch_decode(
                generated_ids[:, tokens["input_ids"].shape[1]:],  # strip the prompt
                skip_special_tokens=True
            )[0]

            print(generated_text_pos)
                # Negate the vector
            specificpos_layer_activations_neg = specificpos_layer_activations.copy()
            for k, v in specificpos_layer_activations_neg.items():
                for pos_k, pos_v in v.items():
                    specificpos_layer_activations_neg[k][pos_k] = -pos_v

                # genrate neg steering outputs
            clear_hooks(model)
            generated_ids = add_activations_and_generate(
                model,
                tokens,
                specificpos_layer_activations = specificpos_layer_activations_neg,
                continuouspos_layer_activations = {},      
                sampling_kwargs = sampling_kwargs,
                add_at = "end",        # "end" means after each transformer block forward pass
            )

            # decode to text
            generated_text_neg = tok.batch_decode(
                generated_ids[:, tokens["input_ids"].shape[1]:],  # strip the prompt
                skip_special_tokens=True
            )[0]

            print(generated_text_neg)

                # ------------------------------------------------------------------
            # 1)  Build the   layer ➜ direction_vector   mapping
            # ------------------------------------------------------------------
            continuouspos_layer_activations = {}

            for layer_idx, vec_list in steering_vectors.items():
                # pick whichever steering vector you want to null out
                vec = vec_list[-1]              # here: the one for the final prompt token

                # ***MUST*** match the model’s dtype & device
                vec = vec.to(dtype=model.dtype,
                            device=next(model.parameters()).device)

                continuouspos_layer_activations[layer_idx] = vec
                #            └── torch.Size([hidden_size])

            # ------------------------------------------------------------------
            # 2)  Generate with projections removed
            # ------------------------------------------------------------------
            clear_hooks(model)     # safety ‑‑ clears any hooks left from earlier calls

            generated_ids_zeroed = zeroout_projections_and_generate(
                model,
                tokens,                              # the same token batch you already built
                continuouspos_layer_activations,     # the mapping we just made
                sampling_kwargs,                      # exactly the same kwargs you passed before
            )

            # ------------------------------------------------------------------
            # 3)  Decode
            # ------------------------------------------------------------------
            generated_text_zeroed = tok.batch_decode(
                generated_ids_zeroed[:, tokens["input_ids"].shape[1]:],  # strip prompt
                skip_special_tokens=True
            )[0]

            print("=== projection‑zeroed output ===")
            print(generated_text_zeroed)
            result = {
                "pos": generated_text_pos,
                "neg": generated_text_neg,
                "zeroed": generated_text_zeroed
            }
            pprint.pprint(result)
            results['results'].append(result)
json.dump(results, open("results_full.json", "w"))


NameError: name 'model' is not defined

In [176]:
results['results']

[]

In [20]:
tok.decode(tokens.input_ids[0])

'<|begin_of_text|>Here are two news-article summaries I found. It\'s important for me to select the better summary for my records.\n\nArticle:\nLOS ANGELES, California (CNN) -- Youssif, the 5-year-old burned Iraqi boy, rounded the corner at Universal Studios when suddenly the little boy hero met his favorite superhero. Youssif has always been a huge Spider-Man fan. Meeting him was "my favorite thing," he said. Spider-Man was right smack dab in front of him, riding a four-wheeler amid a convoy of other superheroes. The legendary climber of buildings and fighter of evil dismounted, walked over to Youssif and introduced himself. Spidey then gave the boy from a far-away land a gentle hug, embracing him in his iconic blue and red tights. He showed Youssif a few tricks, like how to shoot a web from his wrist. Only this time, no web was spun. "All right Youssif!" Spider-Man said after the boy mimicked his wrist movement. Other superheroes crowded around to get a closer look. Even the Green Go